# kubernetes viz

In [ ]:
nMasters = 1        # k8s/kubeflow
nNodes = 2
nWorkersPerNode = 16 # dgx-station

widthPixels = 500
verticalGap = 5; zOffset = 0;

masterXOffset = int( (widthPixels  / nMasters) ) 
nodeXOffset = int( masterXOffset / nNodes )
workerXOffset = int( nodeXOffset / nWorkersPerNode )

In [ ]:
masterPositions = nodePositions = workerPositions = None
masterNodeConnections = nodeWorkerConnections = None

ipv.figure(width=widthPixels, height=500)

for iMaster in range( nMasters ):
    verticalLayerCount = 0
    iMasterOffset = iMaster * masterXOffset
    iMasterPos = np.array( [iMasterOffset, verticalGap * verticalLayerCount, zOffset] ).astype(np.float32)
    
    # append node to position stack
    if masterPositions is None: masterPositions = iMasterPos
    else: masterPositions = np.vstack( ( masterPositions, iMasterPos ) ) 
    masterNodeConnections = None
    
    # nodes
    for iNode in range( nNodes ):     
        verticalLayerCount += 1
        iNodeOffset = iMasterOffset + iNode * nodeXOffset
        iNodePos = np.array( [iNodeOffset, verticalGap * verticalLayerCount, zOffset] ).astype(np.float32)
        
        # append node to position stack
        if nodePositions is None: nodePositions = iNodePos
        else: nodePositions = np.vstack( ( nodePositions, iNodePos ) ) 
        nodeWorkerConnections = None
        
        # workers
        for iWorker in range( nWorkersPerNode ):
            verticalLayerCount += 1
            iWorkerOffset = iNodeOffset + iWorker * workerXOffset
            iWorkerPosition = np.array( [iWorkerOffset, verticalGap * verticalLayerCount, iWorker * 10] ).astype(np.float32)
            
            # append node to position stack
            if workerPositions is None: workerPositions = iWorkerPosition
            else: workerPositions = np.vstack( ( workerPositions, iWorkerPosition ) ) 
            
            # create list of connections between nodes and workers
            if nodeWorkerConnections is None: nodeWorkerConnections = np.vstack( ( iNodePos, iWorkerPosition ) )
            else: nodeWorkerConnections = np.vstack( ( nodeWorkerConnections, np.vstack( ( iNodePos, iWorkerPosition ) ) ) ) 
            
            verticalLayerCount -= 1
        
        ipv.plot(nodeWorkerConnections[:,0],nodeWorkerConnections[:,1],nodeWorkerConnections[:,2], color = 'green')
        
        # create list of connections between master and nodes
        if masterNodeConnections is None: masterNodeConnections = np.vstack( ( iMasterPos, iNodePos ) )
        else: masterNodeConnections = np.vstack( ( masterNodeConnections, np.vstack( ( iMasterPos, iNodePos ) ) ) ) 
        
        verticalLayerCount -= 1
    
    ipv.plot(masterNodeConnections[:,0],masterNodeConnections[:,1],masterNodeConnections[:,2], color = 'red')
    
masterPositions = np.matrix(masterPositions)
nodePositions = np.matrix(nodePositions)
workerPositions= np.matrix(workerPositions)

In [ ]:
def vectorize_input( data ):
    x = np.array( data[:,0] ) if data.shape[0] == 1 else np.array( data[:,0] ).squeeze()
    y = np.array( data[:,1] ) if data.shape[0] == 1 else np.array( data[:,1] ).squeeze()
    z = np.array( data[:,2] ) if data.shape[0] == 1 else np.array( data[:,2] ).squeeze()
    return x,y,z

In [ ]:
 x,y,z = vectorize_input ( masterPositions )
ipv.scatter( x,y,z, marker='box', color='blue', size = 15 )

x,y,z = vectorize_input ( nodePositions )
ipv.scatter( x,y,z, marker='box', color='red', size = 10 )

x,y,z = vectorize_input ( workerPositions )
ipv.scatter( x,y,z, marker='box', color='green', size = 5 )

#ipv.plot(nodeWorkerConnections[:,0],nodeWorkerConnections[:,1],nodeWorkerConnections[:,2], color = 'green')
ipv.pylab.style.box_off()
ipv.pylab.style.axes_off()
#ipv.pylab.style.set_style_dark()
container = ipv.gcc()
container.layout.align_items = 'center'
display(container)